# Lecture Plan

1. AutoGen
2. AutoGen Studio
3. LangSmith
4. other frameworks:
  - Flowise
  - LangFlow

---

### LangChain vs AutoGen


**LangChain:**

1. LLM based applications with Chains, Tools and Agents
2. LangChain natievly supports to multiple external data sources
3. LangChain uses a Chain Based approach. 
4. LangChain recently launched LangGraph which supports DAG in Chains and extends LangChain's existing features to Multi Agent workflows.

**AutoGen**

1. Multi Agent Framework from Microsoft.
2. Used to build LLM Applications with workflows involving the Multi Agent Conversations.
3. AutoGen uses an Agent Based  Approach
4. AutoGen recently launched AutoGen studio. It is a web interface for rapid prototyping.

![AutoGen Framework](./images/autogen.png)


In [ ]:
! pip install pyautogen

#### Config List

- specify the model and other configuration info to agent.

**ways to generate config list:**

1. `get_config_list`: generates config list for API calls
2. `config_list_openai_aoai`:  generates config list for OpenAI and Azure OpenAI endpoints
3. `config_list_from_json`: loads configuration from a JSON file. `filter_dict` is used to filter the `config_list_from_json` file.
4. `config_list_from_models`: creates configurations based on provided list of models.
5. `config_list_from_dotenv`: loads configuration from a .env file. `filter_dict` is used to filter the `config_list_from_json` file.

In [ ]:
import autogen

In [8]:
from autogen import config_list_from_dotenv

config_list = config_list_from_dotenv(
    dotenv_file_path='C:/TrainingMaterial/generative-ai/genai-material/.env',
    filter_dict={
        "model":{
            "gpt-3.5-turbo"
        }
    }
)

#### Agents

AutoGen provides an abstraction for creating the agents with following features:

- **Conversable:** Any agent can send or recieve messages from the other agent.
- **Customizable:** Agents in autogen can be integrated with LLMs, tools and humans.

#### Multi Agent Conversational Framework

![AutoGen Agents](./images/autogen_agents.png)


1. `ConversableAgent`: Genric class for agents that converse with each other by exchnaging messages.
2. `AssistantAgent`: designed to act as an AI Assistant using LLMs.
3. `UserProxyAgent`: proxy for humans.
4. `GroupChatManager` : A chat manager that can manage a group chat of multiple agents.

In [14]:
# Code generation agent

from autogen import AssistantAgent, UserProxyAgent

assistant = AssistantAgent(
    name = "assitant",
    llm_config={
        "config_list": config_list,
        "temperature": 0.0,
        "cache_seed": 42

    },
    max_consecutive_auto_reply= 10
)


user_proxy = UserProxyAgent(

    name = "user_proxy",
    system_message="reply TERMINATE when you are done with the code",
    human_input_mode="TERMINATE",
    code_execution_config={
        "work_dir": "C:/TrainingMaterial/generative-ai/genai-material/trng-1855/trng-git-repo/trng-1855/week-7/",
        "use_docker" : False
    },
    is_termination_msg = lambda x : x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply= 10
    
)

response = user_proxy.initiate_chat(
    assistant,
    message = """Code to sort the numbers using merge sort""",
    summry_method="reflection_from_llms"
)






user_proxy (to assitant):

Code to sort the numbers using merge sort

--------------------------------------------------------------------------------


assitant (to user_proxy):

Here is the Python code to implement the merge sort algorithm for sorting numbers:

```python
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]
    
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)
    
    return merge(left_half, right_half)

def merge(left, right):
    result = []
    i = j = 0
    
    while i < len(left) and j < len(right):
        if left[i] < right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    
    while i < len(left):
        result.append(left[i])
        i += 1
    
    while j < len(right):
        result.append(right[j])
        j += 1
    
    return result

# Test the merge_sort function
numbers = [5, 2, 8, 12, 3, 7]
sorted_numbers = merge_sort(numbers)
print(sorted_numbers)
```

This code defines two functions: `merge_sort` 

user_proxy (to assitant):

exitcode: 0 (execution succeeded)
Code output: 
[2, 3, 5, 7, 8, 12]


--------------------------------------------------------------------------------
assitant (to user_proxy):

Great! The code executed successfully and sorted the numbers using the merge sort algorithm. The output `[2, 3, 5, 7, 8, 12]` confirms that the numbers have been sorted in ascending order.

Is there anything else I can help you with?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
user_proxy (to assitant):



--------------------------------------------------------------------------------
assitant (to user_proxy):

If you have any more questions or need assistance with anything else, please let me know. I'm here to help!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
user_proxy (to assitant):



---------------------------------------------------------------

In [ ]:
import openai

result = openai.completions.create(
    
)

In [ ]:
# Agent Group Chat



#### References


- [AutoGen Docs](https://microsoft.github.io/autogen/docs/Getting-Started)
- [AutoGen SDK](https://microsoft.github.io/autogen/docs/reference/agentchat/conversable_agent)

### AutoGen Studio

- An inteface by AutoGen used to rapildly create agents and multi agent workflows.



In [ ]:
! pip install autogenstudio

`autogenstudio ui --port 8081`